In [1]:
import numpy as np
from joblib import Parallel, delayed
import pandas as pd
import pickle
import lightgbm as lgb
import glob
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from joblib import Parallel, delayed
import time
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import datetime
from sklearn.linear_model import Ridge, LinearRegression
import uqer
from uqer import DataAPI   #优矿api
client = uqer.Client(token='18266a7c0ac9f8cdbe00f9b2ecb65f42316a5f78d9cc22ebabcbd923593356e4')

127807@wmcloud.com 账号登录成功


In [2]:
def ticekrToStr(job_sum):
    job_sum = job_sum[job_sum.ticker<700000]
    job_sum.loc[job_sum.ticker<10,'temp']='00000'
    job_sum.loc[(job_sum.ticker<100)&(job_sum.ticker>=10),'temp']='0000'
    job_sum.loc[(job_sum.ticker<1000)&(job_sum.ticker>=100),'temp']='000'
    job_sum.loc[(job_sum.ticker<10000)&(job_sum.ticker>=1000),'temp']='00'
    job_sum.loc[job_sum.temp==job_sum.temp,'ticker'] = job_sum[job_sum.temp==job_sum.temp]['temp']+job_sum[job_sum.temp==job_sum.temp]['ticker'].astype(str)
    del job_sum['temp']
    job_sum['ticker'] = job_sum['ticker'].astype(str)
    return job_sum

def reduce_mem_usage(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
df1 = pd.read_csv('pct1_cal/alter_idst_alphas_066_labels_raw.csv')
print(df1.shape)
f_index = ['ticker', 'tradeDate']
f_industry = pickle.load(open("pct1_cal/f_industry", "rb"))

(4745949, 327)


In [4]:
df1.columns[:-10]

Index(['ticker', 'tradeDate', 'minute_new_FACTORS_CDPDP',
       'minute_new_FACTORS_CDPP', 'minute_new_FACTORS_Down_beta',
       'minute_new_FACTORS_ILLIQ', 'minute_new_FACTORS_IMI',
       'minute_new_FACTORS_PVC', 'minute_new_FACTORS_TMA_turn',
       'minute_new_FACTORS_TSMOM',
       ...
       'idst_12', 'idst_13', 'idst_14', 'idst_15', 'idst_16', 'idst_17',
       'idst_18', 'idst_19', 'idst_20', 'idst_21'],
      dtype='object', length=317)

In [5]:
print('========LOADING DATA========')
jiheyingzi = pd.read_csv('pct1_cal/集合竞价因子.csv')
jiheyingzi = jiheyingzi.drop(columns='Unnamed: 0')
print(jiheyingzi.shape)
print('========COMPLETE LOADING DATA========')

========LOADING DATA========
(3320037, 12)
========COMPLETE LOADING DATA========


In [6]:
df1 = df1.merge(jiheyingzi, on=['ticker', 'tradeDate'], how='left')
extra_factors = ['volume', 'cachgPct', 'thecommittee', 'askVolume1',
       'bidVolume1', 'caQrr', 'caTr', 'OCVP1',
       'Open/vwap-1', 'Gap']
factors = pickle.load(open("pct1_cal/f_x_066", "rb"))

In [7]:
f_x = extra_factors.copy()
for element in factors:
    if element[:14] == 'py_cov_FACTORS' or element[:14] == 'minute_FACTORS' or element[:14] == 'gaopin_liangji':
        f_x.append(element)
    elif element[:5] == 'alpha':
        f_x.append(element)

In [8]:
# %% a list of classes
## exclude extreme values
class ExcludeExtreme(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.df_median = None
        self.df_standard = None

    def fit(self, df_columnList):
        self.df_median = df_columnList.median()
        self.df_standard = df_columnList.apply(lambda x: x - self.df_median[x.name]).abs().median()
        return self

    def scaller(self, x):
        self.di_max = self.df_median[x.name] + 5 * self.df_standard[x.name]
        self.di_min = self.df_median[x.name] - 5 * self.df_standard[x.name]
        x = x.apply(lambda v: self.di_min if v < self.di_min else v)
        x = x.apply(lambda v: self.di_max if v > self.di_max else v)
        return x

    def transform(self, df_columnList):
        df_columnList = df_columnList.apply(self.scaller)
        return df_columnList

## fill empty values
class FillEmpty(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.df_mean_industry = None

    def fit(self, df_name, f_x = f_x, group_name='industry'):
        self.df_mean_industry = df_name.groupby(group_name).mean()[f_x]
        self.df_mean_industry = self.df_mean_industry.fillna(self.df_mean_industry.mean())
        self.df_mean_industry.columns = [x + '_mean' for x in self.df_mean_industry.columns]
        self.df_mean_industry = self.df_mean_industry.reset_index()
        self.df_mean_industry = self.df_mean_industry.fillna(0)  # 可以删除
        return self

    def transform(self, df_name, f_x = f_x, group_name='industry'):
        df_name_mean = df_name.merge(self.df_mean_industry, on=group_name, how='left')
        df_name_mean[f_x] = df_name_mean[f_x].apply(lambda x: x.fillna(df_name_mean[x.name + '_mean']))
        df_name = df_name_mean[df_name.columns]
        return df_name

## neutralize industry
class Neutralization(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.models = {}

    def regr_fit(self, X, y):
        self.models[y.name] = linear_model.LinearRegression().fit(X, y)
        return

    def regr_pred(self, X, y):
        pred = self.models[y.name].predict(X)
        return y - pred

    def fit(self, df_name, f_x = f_x, f_idst = f_industry):
        X = df_name[['log_marketValue'] + f_idst]
        df_name[f_x].apply(lambda y: self.regr_fit(X, y))
        return self

    def transform(self, df_name, f_x = f_x, f_idst = f_industry):
        X = df_name[['log_marketValue'] + f_idst]
        df_name[f_x] = df_name[f_x].apply(lambda y: self.regr_pred(X, y))
        return df_name

In [9]:
label_list = ['PCT5_rank', 'PCT2_rank', 'openclose_pct1_rank', 'openclose_pct1', 'askbid_pct1_rank', 'askbid_pct1']
f_y =  label_list[2] # 'pct_dmean'
f_x = pickle.load(open("pct1_cal/f_x_066", "rb")) + extra_factors

#e: extra, m: filted
data_source = 'full_Alter_066_full_extra'
file_location = 'pct1_cal'
file_type = 'full'
model_list = ['LinearRegression', 'RidgeR', 'DecisionTreeR', 'XGBoostR', 'LGBMRegressor']
model_name = model_list[4]
target_types = ['r', 'c'] # 分类问题还是回归问题 r 回归问题 c 分类问题
target_type = target_types[0]
####========需要修改的全局参数========####

In [10]:
dates = df1.tradeDate.sort_values().unique()
epoch_ts = list(dates)
f_index = ['ticker', 'tradeDate']
result_name = '{}_{}_{}_{}'.format(data_source, model_name, f_y, target_type)
print(result_name)

update = 22 # 训练长度：22天
train_si = epoch_ts.index('2017-01-03') # included. '2017-01-03'
train_ei = epoch_ts.index('2019-01-02') # excluded. '2018-12-28'
test_si = epoch_ts.index('2019-01-02') # included. '2019-01-02'
test_ei = epoch_ts.index('2019-02-01') # excluded. '2019-01-31' '2019-02-01'
test_fi = len(epoch_ts) - 1 # excluded. '2019-01-16'

# number of epochs，循环次数
num_epoch = round((test_fi - test_ei) / update)
epoch_range = range(0, num_epoch + 1)
timediff = pd.Timedelta(100,unit='d')

full_Alter_066_full_extra_LGBMRegressor_openclose_pct1_rank_r


In [11]:
#每日rank
def dailyrank(data,out):
    datelist = np.unique(data['tradeDate'])
    data['rank'] = 0
    for i in datelist:
        data.loc[data['tradeDate']==i,'rank'] = data.loc[data['tradeDate']==i,out].rank(ascending = True)/data.loc[data['tradeDate']==i,:].shape[0]
    return data

In [12]:
def load_lrlist(lr_max,lr_min,num_rounds):
    lrlist = [lr_max+(lr_min-lr_max)*(np.log(i)/np.log(num_rounds)) for i in range(1,num_rounds+1)]
    return lrlist

In [13]:
lr_max = 0.05
lr_min = 0.02
n_jobs = 20

In [14]:
#gbm params
params =  {
    'boosting_type' : 'gbdt',
    'num_leaves' : 31,
    'feature_fraction' : 0.8,
    'bagging_fraction' : 0.8,
    'lambda_l1':1,
    'lambda_l2':10,
    'max_bin' : 64,
    'num_boost_round': 200,
    'learning_rate' : 0.04,
    'min_data_in_leaf':10,
    'num_threads': n_jobs
}
params['objective'] = 'regression'
params['metric'] = {'l2', 'auc'}
lrlist = load_lrlist(lr_max,lr_min,params['num_boost_round'])
params['lrlist'] = lrlist

In [15]:
def assign_weight(trainy,weightdic):
    trainy['weight'] = 1
    trainy['absout'] = abs(trainy['openclose_pct1'])
    wkey = [k for k in weightdic]
    wkey.sort()
    for k in wkey[:-1]:
        k2 = wkey[wkey.index(k)+1]
        trainy.loc[(trainy['absout']>=k)&(trainy['absout']<k2),'weight'] *= weightdic[k]
    k = wkey[-1]
    trainy.loc[trainy['absout']>=k,'weight'] *= weightdic[k]
    trainy['weight'] = (trainy['weight']*trainy.shape[0])/(trainy['weight'].sum())
    del(trainy['absout'])
    return trainy

In [16]:
weightdic = {0.08:3, 0.05:2, 0.01:1.5}

In [17]:
df_result_all = pd.DataFrame()
total_IC = 0
start = time.time()
for epoch in epoch_range:
    print('----- EPOCH {}------'.format(epoch))
    update_n = epoch * update
    # get a list of train dates
    epoch_t_train = epoch_ts[train_si + update_n : train_ei + update_n]
    # get a list of test dates
    epoch_t_test = epoch_ts[test_si + update_n : test_ei + update_n]
    df_train = df1[df1.tradeDate.apply(lambda x: x in epoch_t_train)].reset_index(drop=True)
    df_test = df1[df1.tradeDate.apply(lambda x: x in epoch_t_test)].reset_index(drop=True)
    print('预测时间：', epoch_t_test)
    print('数据大小：', df_train.shape, df_test.shape)


    print('处理前无效值: train{}, test{}'.format(df_train[f_y].isnull().any().sum(), 
    df_test[f_y].isnull().any().sum()))
    df_train = df_train[~df_train.log_marketValue.isnull()]
    df_train = df_train[~df_train.openclose_pct1_rank.isnull()]
    df_test = df_test[~df_test.log_marketValue.isnull()]
    df_test = df_test[~df_test.openclose_pct1_rank.isnull()]
    print('处理后无效值: train{}, test{}'.format(df_train[f_y].isnull().any().sum(), 
    df_test[f_y].isnull().any().sum()))

    # 中位数去极值
    print('exclude extreme values')
    mevtransformer = ExcludeExtreme()
    mevtransformer.fit(df_train[f_x])
    df_train[f_x] = mevtransformer.transform(df_train[f_x])
    df_test[f_x] = mevtransformer.transform(df_test[f_x])

    # 缺失值处理
    print('deal with null values')
    gvfiller = FillEmpty()
    gvfiller = gvfiller.fit(df_train)
    df_train = gvfiller.transform(df_train)
    df_test = gvfiller.transform(df_test)
    print('缺失值: train{}, test{}'.format(df_train[f_x].isnull().any().sum(), df_test[f_x].isnull().any().sum()))

    # 行业中性处理
    print('do neutralization')
    idst_neutral = Neutralization()
    idst_neutral = idst_neutral.fit(df_train)
    df_train = idst_neutral.transform(df_train)
    df_test = idst_neutral.transform(df_test)


    # 标准化处理
    print('do standardization')
    scaler = StandardScaler()
    scaler = scaler.fit(df_train[f_x])
    df_train[f_x] = scaler.transform(df_train[f_x])
    df_test[f_x] = scaler.transform(df_test[f_x])


    # 获得 x
    x_train = df_train[f_x].values
    x_test = df_test[f_x].values
    print('处理后x:', x_train.shape, x_test.shape)

    # 获得y
    y_train = df_train[['openclose_pct1_rank']]
    y_test = df_test[['openclose_pct1_rank']]
    print('处理后y:', y_train.shape, y_test.shape)
    lgb_train = lgb.Dataset(x_train, y_train[f_y].values)
    model = lgb.train(params,
                    lgb_train,
                    num_boost_round=params['num_boost_round'],
                    callbacks = [lgb.reset_parameter(learning_rate = params['lrlist'])]
                    )
    y_pred = model.predict(x_test)

    # 获得结果
    print(f'耗时:{time.time() - start}')
    print('get result')
    df_result = df_test[f_index].copy()
    df_result['y'] = y_test[f_y].values
    df_result['y_pred'] = y_pred
    IC = df_result[['y', 'y_pred']].corr().iloc[0,1]
    print(IC)
    df_result_all = df_result_all.append(df_result)

print(f'耗时:{time.time() - start}') 
print('sort values')
df_result_all = df_result_all.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
IC = df_result_all[['y', 'y_pred']].corr().iloc[0,1]
today = (datetime.datetime.now()).strftime("%Y-%m-%d:%H:%M:%S")
df_result_all.to_csv('{}/{}/{}{}_{}.csv'.format(file_location, file_type, result_name, today, round(IC, 4)), index=False)
print('======== COMPLETED {} {} ========'.format(model_name, IC))


----- EPOCH 0------
预测时间： ['2019-01-02', '2019-01-03', '2019-01-04', '2019-01-07', '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11', '2019-01-14', '2019-01-15', '2019-01-16', '2019-01-17', '2019-01-18', '2019-01-21', '2019-01-22', '2019-01-23', '2019-01-24', '2019-01-25', '2019-01-28', '2019-01-29', '2019-01-30', '2019-01-31']
数据大小： (1589021, 337) (78389, 337)
处理前无效值: train1, test1
处理后无效值: train0, test0
exclude extreme values
deal with null values
缺失值: train136, test119
do neutralization
do standardization


/home/liufengyuan/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:985: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/liufengyuan/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:990: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/home/liufengyuan/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:1020: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction ** 2 / new_sample_count


处理后x: (1542090, 300) (77896, 300)
处理后y: (1542090, 1) (77896, 1)


/home/liufengyuan/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Unknown parameter: lrlist
[LightGBM] [Warning] Unknown parameter: lrlist
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.091810 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 18254
[LightGBM] [Info] Number of data points in the train set: 1542090, number of used features: 291
[LightGBM] [Info] Start training from score 0.500157
耗时:1253.115818977356
get result
0.20530077848444458
----- EPOCH 1------


...

/home/liufengyuan/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Unknown parameter: lrlist
[LightGBM] [Warning] Unknown parameter: lrlist
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.099852 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 18378
[LightGBM] [Info] Number of data points in the train set: 1670031, number of used features: 293
[LightGBM] [Info] Start training from score 0.500147
耗时:13065.148656129837
get result
0.1873729934407313
----- EPOCH 10------


/tmp/ipykernel_4464/734324227.py:82: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result_all = df_result_all.append(df_result)


预测时间： ['2019-11-28', '2019-11-29', '2019-12-02', '2019-12-03', '2019-12-04', '2019-12-05', '2019-12-06', '2019-12-09', '2019-12-10', '2019-12-11', '2019-12-12', '2019-12-13', '2019-12-16', '2019-12-17', '2019-12-18', '2019-12-19', '2019-12-20', '2019-12-23', '2019-12-24', '2019-12-25', '2019-12-26', '2019-12-27']
数据大小： (1712395, 337) (82088, 337)
处理前无效值: train1, test1
处理后无效值: train0, test0
exclude extreme values
deal with null values
缺失值: train133, test116
do neutralization


KeyboardInterrupt: 